In [1]:
import json

In [2]:
from biaffine import BiaffineModel

In [3]:
class Args(object):
    def __init__(self, *initial_data, **kwargs):
        for dictionary in initial_data:
            for key in dictionary:
                setattr(self, key, dictionary[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])

In [4]:
args = Args(bert_model_path='/root/autodl-nas/pretrain-models/roberta-base',bert_feature_dim=768,biaffine_size=300,class_num=8,max_sequence_len=128)

In [5]:
model = BiaffineModel(args)

Some weights of the model checkpoint at /root/autodl-nas/pretrain-models/roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import torch

In [7]:
model.load_state_dict(torch.load('savemodel/triplet_v11.bin'))

<All keys matched successfully>

In [8]:
model.eval()
pass

In [9]:
import nltk

In [10]:
from transformers import AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained(args.bert_model_path,add_prefix_space=True)

In [12]:
def convert(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = ['##'] + tokens
    sen_length = len(tokens)
    token_range = []
    bert_tokens = tokenizer.encode(tokens,is_split_into_words=True)
    length = len(bert_tokens)
    bert_tokens_padding = torch.zeros(args.max_sequence_len).long()
    mask = torch.zeros(args.max_sequence_len).long()

    for i in range(length):
        bert_tokens_padding[i] = bert_tokens[i]
    mask[:length] = 1

    token_start = 1
    for i, w, in enumerate(tokens):
        token_end = token_start + len(tokenizer.encode(w, add_special_tokens=False))
        token_range.append([token_start, token_end-1])
        token_start = token_end
    assert length == token_range[-1][-1]+2
    return bert_tokens_padding,mask,token_range,sen_length,tokens

In [13]:
bert_tokens_padding,mask,token_range,sen_length,tokens = convert('But when I plugged in my MacBook Pro 13" M1 (A2338), my Mac saw power, but it would not charge the battery at all, ever. ')

In [91]:
# bert_tokens_padding,mask,token_range,sen_length,tokens = convert('Good. Shangjin is cool! Shangjin is awsome!')

In [14]:
bert_tokens_padding.shape

torch.Size([128])

In [15]:
mask.dtype

torch.int64

In [16]:
bert_tokens_padding
# tokens
# mask

tensor([    0, 47385,    53,    77,   939, 30251,    11,   127, 13418,  6298,
         1759,   508, 12801,   475,   134,    36,    10,  1922,  3170,  4839,
         2156,   127, 13418,   794,   476,  2156,    53,    24,    74,    45,
         1427,     5,  3822,    23,    70,  2156,   655,   479,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [17]:
preds = model(bert_tokens_padding.unsqueeze(0), mask.unsqueeze(0))

In [18]:
preds = torch.argmax(preds, dim=3)[0]

In [19]:
preds

tensor([[0, 1, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [20]:
ASPECT_BEGIN=1
ASPECT_IN=2
OPINION_BEGIN=3
OPINION_IN=4
PAIR=5
sentiment2id = {'观点-负面': 5, '观点-中性':6, '观点-正面': 7}

ASPECT=[ASPECT_BEGIN,ASPECT_IN]
OPINION=[OPINION_BEGIN,OPINION_IN]

In [21]:
id2sentiment = {v:k for k,v in sentiment2id.items()}

In [22]:
def get_spans(tags, length, token_range, type):
    spans = []
    start = -1
    begin,mid = type
    for i in range(length):
        l, r = token_range[i]
        if tags[l][l] == -1:
            continue
        elif tags[l][l] == begin:
            if start != -1:
                spans.append([start, i - 1])
            start = i
        elif tags[l][l] not in type:
            if start != -1:
                spans.append([start, i - 1])
                start = -1
    if start != -1:
        spans.append([start, length - 1])
    return spans

In [23]:
def find_triplet(tags, aspect_spans, opinion_spans, token_ranges):
    triplets = []
    for al, ar in aspect_spans:
        for pl, pr in opinion_spans:
            tag_num = [0] * 8
            for i in range(al, ar + 1):
                for j in range(pl, pr + 1):
                    a_start = token_ranges[i][0]
                    o_start = token_ranges[j][0]
                    if al < pl:
                        tag_num[int(tags[a_start][o_start])] += 1
                    else:
                        tag_num[int(tags[o_start][a_start])] += 1

            if sum(tag_num[5:]) == 0: continue
            sentiment = -1
            if tag_num[5] >= tag_num[6] and tag_num[5] >= tag_num[7]:
                sentiment = 5
            elif tag_num[6] >= tag_num[5] and tag_num[6] >= tag_num[7]:
                sentiment = 6
            elif tag_num[7] >= tag_num[5] and tag_num[7] >= tag_num[6]:
                sentiment = 7
            if sentiment == -1:
                continue
            triplets.append([al, ar, pl, pr, sentiment])
    return triplets

In [24]:
predicted_aspect_spans = get_spans(preds, sen_length, token_range, ASPECT)
predicted_opinion_spans = get_spans(preds, sen_length, token_range, OPINION)

In [25]:
predicted_aspect_spans,predicted_opinion_spans, sen_length

([[0, 0]], [[23, 27]], 33)

In [26]:
triplets = find_triplet(preds, predicted_aspect_spans,predicted_opinion_spans,token_range)

In [27]:
triplets

[[0, 0, 23, 27, 5]]

In [28]:
for al,ar,pl,pr,sentiment in triplets:
    aspect = ' '.join(tokens[al:ar+1])
    opinion = ' '.join(tokens[pl:pr+1])
    sentiment = id2sentiment[sentiment]
    print((aspect,opinion,sentiment))

('##', 'would not charge the battery', '观点-负面')


### Export

In [29]:
model_id="gts-absa-v11"
save_path="/root/autodl-nas/export_models/absa/"
max_length=128
opset_version=14
export_model_path=f"{save_path}/{model_id}.onnx"

In [30]:
from os import environ
from psutil import cpu_count

# Constants from the performance optimization available in onnxruntime
# It needs to be done before importing onnxruntime
environ["OMP_NUM_THREADS"] = str(cpu_count(logical=True))
environ["OMP_WAIT_POLICY"] = 'ACTIVE'

from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions, get_all_providers

class OnnxModel:
    def __init__(self, model_path: str, provider: str):
        assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"
        self.options = self._set_options()
        self.model=InferenceSession(model_path, self.options, providers=[provider])
        # Load the model as a graph and prepare the CPU backend 
        self.model.disable_fallback()

    def __call__(self, input):
        return self.model.run(None, input)[0]
  
    def _set_options(self):
        # Few properties that might have an impact on performances (provided by MS)
        options = SessionOptions()
        options.intra_op_num_threads = 1
        options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
        return options

In [31]:
class MLC(torch.nn.Module):
    def __init__(self, basemodel):
        super().__init__()
        self.basemodel = basemodel

    def forward(self, input_ids,attention_mask):
        outputs = self.basemodel(input_ids=input_ids,attention_mask=attention_mask)
        logits = outputs
        return torch.reshape(torch.argmax(logits,dim=3),(-1,128,128))

In [32]:
mlc = MLC(model)

In [33]:
paraphrase={'input_ids':bert_tokens_padding.unsqueeze(0), "attention_mask":mask.unsqueeze(0)}

In [34]:
x = mlc(**paraphrase)

In [35]:
# generate input and output names/keys
input_names = list(paraphrase.keys())
output_names = ["output_0"]

# Generate dynamic axes, with batching -> inputs/outputs with potential dynamic shape
symbolic_names = {0: 'batch_size'} #TODO: Exaplain

input_dynamic_axes = {input_key: symbolic_names for input_key in input_names}
output_dynamic_axes = {output_key: {0: 'batch_size'} for output_key in output_names}
dynamic_axes = dict(input_dynamic_axes, **output_dynamic_axes)

print(f"created input_names:")
print(input_names)
print(f"created output_names:")
print(output_names)
print(f"created dynamic_axes:")
print(dynamic_axes)

created input_names:
['input_ids', 'attention_mask']
created output_names:
['output_0']
created dynamic_axes:
{'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'output_0': {0: 'batch_size'}}


In [36]:
dynamic_axes

{'input_ids': {0: 'batch_size'},
 'attention_mask': {0: 'batch_size'},
 'output_0': {0: 'batch_size'}}

In [37]:
import os
import torch

mlc.eval()

with torch.no_grad():
    torch.onnx.export(mlc,                               # model being run
                      args=tuple(paraphrase.values()),     # model input (or a tuple for multiple inputs)
                      f=export_model_path,                 # where to save the model (can be a file or file-like object)
                      opset_version=14,         # the ONNX version to export the model to
                      do_constant_folding=True,            # whether to execute constant folding for optimization
                      enable_onnx_checker=True, 
                      use_external_data_format=False,
                      input_names=input_names,             # the model's input names  'input_ids', 'token_type_ids', 'attention_mask'
                      output_names=output_names,           # the model's output names 'output_0'
                      dynamic_axes=dynamic_axes)           # inputs/outputs with potential dynamic shape -> mostly all

print("Model exported at ", export_model_path)

/root/miniconda3/lib/python3.8/site-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/root/miniconda3/lib/python3.8/site-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "
/root/miniconda3/lib/python3.8/site-packages/torch/onnx/symbolic_helper.py:716: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero 

Model exported at  /root/autodl-nas/export_models/absa//gts-absa-v11.onnx


In [38]:
provider = "CPUExecutionProvider"
onnx_model = OnnxModel("/root/autodl-nas/export_models/absa/gts-absa-v11.onnx", provider)

In [39]:
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in paraphrase.items()}
onnx_model(inputs_onnx)

array([[[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=int64)

In [67]:
import onnx

# Load the ONNX model
model2 = onnx.load("/root/autodl-nas/export_models/absa/gts-absa.onnx")

# Check that the model is well formed
onnx.checker.check_model(model2)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model2.graph))

graph torch-jit-export (
  %input_ids[INT64, batch_sizex128]
  %attention_mask[INT64, batch_sizex128]
) initializers (
  %basemodel.bert.embeddings.word_embeddings.weight[FLOAT, 50265x768]
  %basemodel.bert.embeddings.position_embeddings.weight[FLOAT, 514x768]
  %basemodel.bert.embeddings.token_type_embeddings.weight[FLOAT, 1x768]
  %basemodel.bert.embeddings.LayerNorm.weight[FLOAT, 768]
  %basemodel.bert.embeddings.LayerNorm.bias[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.attention.self.query.bias[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.attention.self.key.bias[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.attention.self.value.bias[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.attention.output.dense.bias[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.attention.output.LayerNorm.weight[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.attention.output.LayerNorm.bias[FLOAT, 768]
  %basemodel.bert.encoder.layer.0.intermediate.dense.bias[FLOAT, 3072]
  %basemodel.bert.encoder.layer.0.ou